# Integrated Python codes

In [94]:
import pandas as pd
import numpy as np
import scipy.stats as sps
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as skl
from sklearn import preprocessing
from statistics import mode
from numpy import median
from sklearn.decomposition import PCA
# import tensorflow and keras for autoencoder neural net work
import pickle
from scipy import stats
import tensorflow as tf
# from sklearn.model_selection import train_test_split
from keras.models import Model, load_model
from keras.layers import Input, Dense
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers
%matplotlib inline

Using TensorFlow backend.


# Import Data

In [2]:
%%time
mydata=pd.read_csv('NY property data.csv')

CPU times: user 4.76 s, sys: 968 ms, total: 5.72 s
Wall time: 5.24 s


# Data Cleaning



`
The data cleaning process was carried out for all the relevant variables that help describe the valuation, area and location of the property.
`

## Fill Missing Values

The process entailed filling missiing values in such a way that it ensures the new values are as close to the actual values as possible and they do not trigger an alarm ini our models as outliers. The process, therefore, was mainly carried out by aggregating all the entries over relevant variables and then filling the missing value by the `median/average/most frequent values` of the missing field within the aggregate that matches with the relevant variables of the given entry.<br> A detailed description of the variables identified and the process to fill in these missing values has been given below:

### Zip Codes

In [3]:
# Define a new function to calculate the most frequent value, because the built-in function cann't solve a tie.
def mods(i):
    d = {}
    for x in i:
        d[x] = d.get(x, 0) +1
   
    return max(d, key = lambda k: d[k])

In [4]:
# Before fill in, we need to get modes for each aggregate/group
# Incase there are insufficient entries in each group
# We prepared a back-up table which is aggregated by one less category

# First, aggregate the records using the Borough, BLOCK, TAXCLASS and compute the mode of the Zip code field
d1=mydata[['ZIP','B','BLOCK','TAXCLASS']]
count=d1.groupby(['B','TAXCLASS','BLOCK']).agg({'ZIP':[mods, 'count']})

# Drop one category and aggregate again to get a higher overall frequency.
count2=d1.groupby(['B','TAXCLASS']).agg({'ZIP':[mods, 'count']})

# Mode for all ZIP
m = mode(mydata['ZIP'])

# When there are multiple levels in column names, merge first and second level
# count.columns = list(map(''.join, count.columns.values))

# Delete the first level in column names
count.columns=count.columns.droplevel()
count2.columns=count2.columns.droplevel()

# Merge tables to mydata
mydata=mydata.merge(count, on = ['B', 'BLOCK','TAXCLASS'])
mydata = mydata.merge(count2, on = ['B','TAXCLASS'],suffixes = ['','_wide'])  # suffix columns in second table

mydata = mydata.rename(columns = {"mods":"mode",'mods_wide':'widemode','count_wide':'widecount'}) 

__Impute missing ZIP values__<br>
The rule is : If the group size is greater than 10 records, then use mode in that group, or if less than 10, use mode obtained by aggregating Borough and TAXCLASS. If the number of records in that new group does not exist, use mode by aggregating just TAXCLASS.

In [5]:
%%time

def compare(x):
    
    if x['present']:
     #   print(2)
        if int(x['count'])>= 10:
            return x['mode']
        
        elif int(x['widecount']) >= 10 and (not np.isnan(x['widemode'])):
            return x['widemode']
        else:
            return m
    return x['ZIP']

# Impute missing values in ZIP
mydata['present'] = mydata['ZIP'].isna()
mydata['ZIPS']  = mydata.apply(compare, axis = 1)

CPU times: user 22.1 s, sys: 1.22 s, total: 23.3 s
Wall time: 23.6 s


In [6]:
# check
mydata[np.isnan(mydata['ZIPS'])][['ZIPS']] 

,ZIPS


In [7]:
mydata['ZIP'] = mydata['ZIPS'].apply(lambda x: str(int(x)))
mydata['ZIP3'] = mydata['ZIP'].apply(lambda x: x[:3])
mydata['ZIP4'] = mydata['ZIP'].apply(lambda x: x[:4])

# Keep useful columns 
# ['RECORD', 'B', 'TAXCLASS', 'LTFRONT', 'LTDEPTH', 'STORIES', 'FULLVAL', 
# 'AVLAND', 'AVTOT', 'ZIP', 'BLDFRONT', 'BLDDEPTH', 'ZIP3', 'ZIP4']

for x in ['BBLE','BLOCK', 'ZIPS','LOT', 'EASEMENT','OWNER','BLDGCL','EXT','EXLAND',
          'EXTOT','EXCD1','STADDR','EXMPTCL','AVLAND2', 'AVTOT2', 'EXLAND2','EXTOT2',
          'EXCD2','PERIOD','YEAR','VALTYPE','mode','count','widemode','widecount','present']:
    del mydata[x] 

### Stories

In [8]:
# Define a new function getting median even though NA exists.
def median_rm_na(x):
    l=[]
    for i in x:
        if i!=0 and np.isnan(i)==False:
            l.append(float(i))
    return np.median(l)

In [9]:
# Similar rules as we get modes
d2=mydata[['TAXCLASS','ZIP','STORIES']]
count=d2.groupby(['ZIP','TAXCLASS']).agg({'STORIES':[median_rm_na, 'count']})
count2=d2.groupby(['TAXCLASS']).agg({'STORIES':[median_rm_na, 'count']})

count.columns=count.columns.droplevel()
count2.columns=count2.columns.droplevel()


mydata = mydata.merge(count, on = ['ZIP','TAXCLASS'])
mydata = mydata.merge(count2, on = ['TAXCLASS'],suffixes = ['', '_wide'])
mydata = mydata.rename(columns = {"median_rm_na":"median",'median_rm_na_wide':'widemedian','count_wide':'widecount'}) 
m = mydata['STORIES'].median()


/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [10]:
def compare(x):
    
    if x['present']:
        if int(x['count'])>= 10:
            return x['median']
        elif int(x['widecount'])>=10 and (not np.isnan(x['widemedian'])):
            return x['widemedian']
        else:
            return m
    return x['STORIES']

mydata['present'] = mydata['STORIES'].isna()
mydata['STORY']  = mydata.apply(compare, axis = 1)

In [11]:
mydata[np.isnan(mydata['STORY'])][['STORY']] 

,STORY


In [12]:
mydata['STORIES']=mydata['STORY']

for x in ['median', 'count', 'widemedian', 'widecount', 'present', 'STORY']:
    del mydata[x]

### FullVal

In [13]:
%%time

d3=mydata[['TAXCLASS','ZIP','ZIP3','FULLVAL']]
count=d3.groupby(['ZIP','TAXCLASS']).agg({'FULLVAL':[median_rm_na, 'count']})
count2=d3.groupby(['ZIP3','TAXCLASS']).agg({'FULLVAL':[median_rm_na,'count']})
# After groupping by zip3 and taxclass, there are some groups have less than 5 records and the median is NaN
# Thus, group by taxclass only to avoid NaN
count3=d3.groupby(['TAXCLASS']).agg({'FULLVAL':[median_rm_na,'count']})

count.columns=count.columns.droplevel()
count2.columns=count2.columns.droplevel()
count3.columns=count3.columns.droplevel()
m=mydata['FULLVAL'].median()

mydata = mydata.merge(count, on = ['ZIP','TAXCLASS'])
mydata = mydata.merge(count2, on = ['ZIP3','TAXCLASS'],suffixes = ['', '_wide'])
mydata = mydata.merge(count3, on = ['TAXCLASS'],suffixes= ['','_wider'])

mydata=mydata.rename(columns={'median_rm_na':'median','median_rm_na_wide':'widemedian',
                'median_rm_na_wider':'widermedian','count_wide':'widecount','count_wider':'widercount'})


# new function to impute missing values
def compare(x):
    
    if (x['present']==True) or (float(x['FULLVAL'])==0):
     #   print(2)
        if int(x['count'])>= 10 and (not np.isnan(x['median'])):
            return x['median']
        elif int(x['widecount'])>=10 and (not np.isnan(x['widemedian'])):
            return x['widemedian']
        elif int(x['widercount'])>=10 and (not np.isnan(x['widermedian'])):
            return x['widermedian']
        else:
            return m
    return x['FULLVAL']
mydata['present'] = mydata['FULLVAL'].isna()
mydata['FULLVAL2']  = mydata.apply(compare, axis = 1)
mydata[np.isnan(mydata['FULLVAL2'])][['FULLVAL2']] 

CPU times: user 41.4 s, sys: 1.51 s, total: 42.9 s
Wall time: 36.8 s


In [14]:
mydata['FULLVAL']=mydata['FULLVAL2']
for x in ['median', 'count', 'widecount','widercount','widemedian', 'widermedian', 'present','FULLVAL2']:
    del mydata[x]

### AVLAND

In [15]:
d4=mydata[['TAXCLASS','ZIP','ZIP3','AVLAND']]
count=d4.groupby(['ZIP','TAXCLASS']).agg({'AVLAND':[median_rm_na, 'count']})
count2=d4.groupby(['ZIP3','TAXCLASS']).agg({'AVLAND':[median_rm_na,'count']})
# After groupping by zip3 and taxclass, there are some groups have less than 5 records and the median is NaN
# Thus, group by taxclass only to avoid NaN
count3=d4.groupby(['TAXCLASS']).agg({'AVLAND':[median_rm_na,'count']})

count.columns=count.columns.droplevel()
count2.columns=count2.columns.droplevel()
count3.columns=count3.columns.droplevel()
m=mydata['AVLAND'].median()
mydata = mydata.merge(count, on = ['ZIP','TAXCLASS'])
mydata = mydata.merge(count2, on = ['ZIP3','TAXCLASS'],suffixes = ['', '_wide'])
mydata = mydata.merge(count3, on = ['TAXCLASS'],suffixes = ['','_wider'])

mydata=mydata.rename(columns={'median_rm_na':'median','median_rm_na_wide':'widemedian',
                'median_rm_na_wider':'widermedian','count_wide':'widecount','count_wider':'widercount'})

def compare(x):
    
    if (x['present']==True) or (float(x['AVLAND'])==0):
     #   print(2)
        if int(x['count'])>= 10 and (not np.isnan(x['median'])):
            return x['median']
        elif int(x['widecount'])>=10 and (not np.isnan(x['widemedian'])):
            return x['widemedian']
        elif int(x['widercount'])>=10 and (not np.isnan(x['widermedian'])):
            return x['widermedian']
        else:
            return m
    return x['AVLAND']

mydata['present'] = mydata['AVLAND'].isna()

mydata['AVLAND2']  = mydata.apply(compare, axis = 1)

In [16]:
mydata[np.isnan(mydata['AVLAND2'])][['AVLAND2']] 

,AVLAND2


In [17]:
mydata['AVLAND']=mydata['AVLAND2']

In [18]:
for x in [ 'median', 'count','widecount','widercount', 'widemedian', 'widermedian', 'present', 'AVLAND2']:
    del mydata[x]

### AVTOT

In [19]:
d5=mydata[['TAXCLASS','ZIP','ZIP3','AVTOT']]
count=d5.groupby(['ZIP','TAXCLASS']).agg({'AVTOT':[median_rm_na, 'count']})
count2=d5.groupby(['ZIP3','TAXCLASS']).agg({'AVTOT':[median_rm_na,'count']})
# After groupping by zip3 and taxclass, there are some groups have less than 5 records and the median is NaN
# Thus, group by taxclass only to avoid NaN
count3=d5.groupby(['TAXCLASS']).agg({'AVTOT':[median_rm_na,'count']})

count.columns=count.columns.droplevel()
count2.columns=count2.columns.droplevel()
count3.columns=count3.columns.droplevel()
m=mydata['AVLAND'].median()
mydata = mydata.merge(count, on = ['ZIP','TAXCLASS'])
mydata = mydata.merge(count2, on = ['ZIP3','TAXCLASS'],suffixes = ['', '_wide'])
mydata = mydata.merge(count3, on = ['TAXCLASS'],suffixes = ['','_wider'])

mydata=mydata.rename(columns={'median_rm_na':'median','median_rm_na_wide':'widemedian',
                'median_rm_na_wider':'widermedian','count_wide':'widecount','count_wider':'widercount'})


def compare(x):
    
    if (x['present']==True) or (float(x['AVTOT'])==0):
     #   print(2)
        if int(x['count'])>= 10 and (not np.isnan(x['median'])):
            return x['median']
        elif not np.isnan(x['widermedian']):
            return x['widermedian']
        elif not np.isnan(x['widestmedian']):
            return x['widestmedian']
        else:
            return m
    return x['AVTOT']

mydata['present'] = mydata['AVTOT'].isna()
mydata['AVTOT2']  = mydata.apply(compare, axis = 1)

In [20]:
mydata[np.isnan(mydata['AVTOT2'])][['AVTOT2']] 

,AVTOT2


In [21]:
mydata['AVTOT']=mydata['AVTOT2']
for x in [ 'median', 'count', 'widecount','widercount','widemedian', 'widermedian', 'present', 'AVTOT2']:
    del mydata[x]

### LTFRONT

In [22]:
d6=mydata[['TAXCLASS','ZIP','B','LTFRONT']]
count=d6.groupby(['ZIP','TAXCLASS']).agg({'LTFRONT':[median_rm_na, 'count']})
count2=d6.groupby(['B','TAXCLASS']).agg({'LTFRONT':[median_rm_na,'count']})

count.columns=count.columns.droplevel()
count2.columns=count2.columns.droplevel()
mydata = mydata.merge(count, on = ['ZIP','TAXCLASS'])
mydata = mydata.merge(count2, on = ['B','TAXCLASS'],suffixes = ['', '_wide'])
m=mydata['LTFRONT'].median()

mydata=mydata.rename(columns={'median_rm_na':'median','median_rm_na_wide':'widemedian','count_wide':'widecount'})

def compare(x):
    
    if (x['present']==True) or (float(x['LTFRONT'])==0):
     #   print(2)
        if int(x['count'])>= 20 and (not np.isnan(x['median'])):
            return x['median']
        elif not np.isnan(x['widemedian']):
            return x['widemedian']
        else:
            return m
    return x['LTFRONT']

mydata['present'] = mydata['LTFRONT'].isna()
mydata['LTFRONT2']  = mydata.apply(compare, axis = 1)

In [23]:
#check
mydata[np.isnan(mydata['LTFRONT2'])][['LTFRONT2']] 

,LTFRONT2


In [24]:
mydata['LTFRONT']=mydata['LTFRONT2']
for x in [ 'median', 'count', 'widemedian', 'widecount','present', 'LTFRONT2']:
    del mydata[x]

### LTDEPTH

In [25]:
d7=mydata[['TAXCLASS','ZIP','B','LTDEPTH']]
count=d7.groupby(['ZIP','TAXCLASS']).agg({'LTDEPTH':[median_rm_na, 'count']})
count2=d7.groupby(['B','TAXCLASS']).agg({'LTDEPTH':[median_rm_na,'count']})

count.columns=count.columns.droplevel()
count2.columns=count2.columns.droplevel()
mydata = mydata.merge(count, on = ['ZIP','TAXCLASS'])
mydata = mydata.merge(count2, on = ['B','TAXCLASS'],suffixes = ['', '_wide'])
m=mydata['LTDEPTH'].median()

mydata=mydata.rename(columns={'median_rm_na':'median','median_rm_na_wide':'widemedian','count_wide':'widecount'})


def compare(x):
    
    if (x['present']==True) or (float(x['LTDEPTH'])==0):
     #   print(2)
        if int(x['count'])>= 20 and (not np.isnan(x['median'])):
            return x['median']
        elif not np.isnan(x['widemedian']):
            return x['widemedian']
        else:
            return m
    return x['LTDEPTH']

mydata['present'] = mydata['LTDEPTH'].isna()
mydata['LTDEPTH2']  = mydata.apply(compare, axis = 1)

In [26]:
#check
mydata[np.isnan(mydata['LTDEPTH2'])][['LTDEPTH2']] 

,LTDEPTH2


In [27]:
mydata['LTDEPTH']=mydata['LTDEPTH2']
for x in [ 'median', 'count', 'widecount','widemedian', 'present', 'LTDEPTH2']:
    del mydata[x]
mydata.head(5)

,RECORD,B,TAXCLASS,LTFRONT,LTDEPTH,STORIES,FULLVAL,AVLAND,AVTOT,ZIP,BLDFRONT,BLDDEPTH,ZIP3,ZIP4
0,1,1,4,500.0,1046.0,50.0,21400000.0,4225500.0,9630000.0,10004,0,0,100,1000
1,2,1,4,27.0,207.0,50.0,193800000.0,14310000.0,87210000.0,10004,0,0,100,1000
2,3,1,4,709.0,564.0,3.0,104686000.0,39008700.0,47108700.0,10004,709,564,100,1000
3,4,1,4,793.0,551.0,2.0,39200000.0,15255000.0,17640000.0,10004,85,551,100,1000
4,5,1,4,323.0,1260.0,1.0,272300000.0,121050000.0,122535000.0,10004,89,57,100,1000


### BLDFRONT

In [28]:
d8=mydata[['TAXCLASS','ZIP','B','BLDFRONT']]
count=d8.groupby(['ZIP','TAXCLASS']).agg({'BLDFRONT':[median_rm_na, 'count']})
count2=d8.groupby(['B','TAXCLASS']).agg({'BLDFRONT':[median_rm_na,'count']})

count.columns=count.columns.droplevel()
count2.columns=count2.columns.droplevel()
mydata = mydata.merge(count, on = ['ZIP','TAXCLASS'])
mydata = mydata.merge(count2, on = ['B','TAXCLASS'],suffixes = ['', '_wide'])
m=mydata['BLDFRONT'].median()

mydata=mydata.rename(columns={'median_rm_na':'median','median_rm_na_wide':'widemedian','count_wide':'widecount'})


def compare(x):
    
    if (x['present']==True) or (float(x['BLDFRONT'])==0):
     #   print(2)
        if int(x['count'])>= 20 and (not np.isnan(x['median'])):
            return x['median']
        elif not np.isnan(x['widemedian']):
            return x['widemedian']
        else:
            return m
    return x['BLDFRONT']

mydata['present'] = mydata['BLDFRONT'].isna()
mydata['BLDFRONT2']  = mydata.apply(compare, axis = 1)

In [29]:
mydata[np.isnan(mydata['BLDFRONT2'])][['BLDFRONT2']] 

,BLDFRONT2


In [30]:
mydata['BLDFRONT']=mydata['BLDFRONT2']
for x in [ 'median', 'count', 'widemedian', 'present', 'BLDFRONT2']:
    del mydata[x]

### BLDDEPTH

In [31]:
d9=mydata[['TAXCLASS','ZIP','B','BLDDEPTH']]
count=d9.groupby(['ZIP','TAXCLASS']).agg({'BLDDEPTH':[median_rm_na, 'count']})
count2=d9.groupby(['B','TAXCLASS']).agg({'BLDDEPTH':[median_rm_na,'count']})

count.columns=count.columns.droplevel()
count2.columns=count2.columns.droplevel()
mydata = mydata.merge(count, on = ['ZIP','TAXCLASS'])
mydata = mydata.merge(count2, on = ['B','TAXCLASS'],suffixes = ['', '_wide'])
m=mydata['BLDDEPTH'].median()

mydata=mydata.rename(columns={'median_rm_na':'median','median_rm_na_wide':'widemedian','count_wide':'widecount'})


def compare(x):
    
    if (x['present']==True) or (float(x['BLDDEPTH'])==0):
     #   print(2)
        if int(x['count'])>= 20 and (not np.isnan(x['median'])):
            return x['median']
        elif not np.isnan(x['widemedian']):
            return x['widemedian']
        else:
            return m
    return x['BLDDEPTH']

mydata['present'] = mydata['BLDDEPTH'].isna()
mydata['BLDDEPTH2']  = mydata.apply(compare, axis = 1)

In [32]:
mydata[np.isnan(mydata['BLDDEPTH2'])][['BLDDEPTH2']] 

,BLDDEPTH2


In [33]:
mydata['BLDDEPTH']=mydata['BLDDEPTH2']
for x in [ 'median', 'count', 'widecount','widemedian', 'present', 'BLDDEPTH2']:
    del mydata[x]

# Variable Creation

- Identifying relevant variables based on the business understanding of the problem
- Modifying these variables to compare them on a level footing, i.e. on a unit measure
- Creating new variables for every field by dividing its value by aggregated value over groups of similar and relevant characteristics

## Identifying relevant variables

The problem at hand deals with identifying properties that have under or over reported the valuation of  their land for tax exemptions, and therefore the key variables of interest that determine the land's value in the dataset are:
* FULLVAL
* AVLAND
* AVTOT

## Modifying these relevant variables

The value of every property will be a function of its land area, the lot frontage outside the property. One can expect the property to have a higher value if it's based over a large area. Therefore, if these property values are directly used in our model to identify outliers, it is very much possible that the model will get incorrectly biased towards identifying larger properties as outliers.
<br>Therefore, to scale them on a level footing, the values were modified by dividing them with relevant area metrics to calculate the valuation per unit area(sqft).
<br>
- r1-Value of the property per unit lot area
- r2-Value of the property per unit building land area
- r3-Value of the property per unit total area of the building
- r4-Assessed value of the land per unit lot area
- r5-Assessed value of the land per unit building land area
- r6-Assessed value of the land per unit total area of the building
- r7-Total assessed value of the property per unit lot area.
- r8-Total assessed value of the property per unit building area.
- r9-Total assessed value of the property per unit total area of the building

<img src="ratios.png" alt="Alt text that describes the graphic" title="Title text" />

## Create new variables
Now to compare how different these valuations over properties with similar characteristics are, the team identified certain characteristics of interest where comparisons can be made. Those of interest in the dataset being:
1. ZIP5
2. ZIP3
3. TAXCLASS
4. Borough
5. ALL
<br>

For each property now, we calculate 45 ratios of r1-r9 with respect to the aggregated average values of r1-r9 over the five classes defined above based on where they fall.

In [73]:
df=mydata

In [74]:
df['S1'] = df['LTFRONT']* df['LTDEPTH']
df['S2'] = df['BLDFRONT']* df['BLDDEPTH']
df['S3'] = df['S2'] * df['STORIES']
df['V1'] = df['FULLVAL']
df['V2'] = df['AVLAND']
df['V3'] = df['AVTOT']

df['R1'] = df['V1']/df['S1']
df['R2'] = df['V1']/df['S2']
df['R3'] = df['V1']/df['S3']
df['R4'] = df['V2']/df['S1']
df['R5'] = df['V2']/df['S2']
df['R6'] = df['V2']/df['S3']
df['R7'] = df['V3']/df['S1']
df['R8'] = df['V3']/df['S2']
df['R9'] = df['V3']/df['S3']

fraud = pd.DataFrame()
fraud  = df[['RECORD']]

fraud['REC'] = df['RECORD']
df = df[['R1', 'R2', 'R3',\
       'R4', 'R5', 'R6', 'R7', 'R8', 'R9','ZIP', 'B','TAXCLASS', 'RECORD','ZIP3', 'ZIP4' ]]
fraud = fraud.set_index(keys = 'REC')
df = df.set_index(keys = 'RECORD')
fraud = fraud.sort_index()
df = df.sort_index()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


## 45 new variables



In [75]:
a = df.groupby(by = ['ZIP']).agg({'R1':'mean', 'R2':'mean', 'R3':'mean', \
                             'R4':'mean', 'R5':'mean', 'R6':'mean',\
                             'R7':'mean', 'R8':'mean', 'R9':'mean'})

a = df.merge(a, how = 'left', on = 'ZIP', suffixes = ['_x', '_y'])
a.index = range(1,len(a)+1)
fraud['R1Z'] = a['R1_x']/a['R1_y'] 
fraud['R2Z'] = a['R2_x']/a['R2_y'] 
fraud['R3Z'] = a['R3_x']/a['R3_y'] 
fraud['R4Z'] = a['R4_x']/a['R4_y'] 
fraud['R5Z'] = a['R5_x']/a['R5_y'] 
fraud['R6Z'] = a['R6_x']/a['R6_y'] 
fraud['R7Z'] = a['R7_x']/a['R7_y'] 
fraud['R8Z'] = a['R8_x']/a['R8_y'] 
fraud['R9Z'] = a['R9_x']/a['R9_y'] 
fraud = fraud.sort_index()
df = df.sort_index()


In [76]:
a = df.groupby(by = ['B']).agg({'R1':'mean', 'R2':'mean', 'R3':'mean', \
                             'R4':'mean', 'R5':'mean', 'R6':'mean',\
                             'R7':'mean', 'R8':'mean', 'R9':'mean'})

a = df.merge(a, how = 'left', on = 'B', suffixes = ['_x', '_y'])
a.index = range(1,len(a)+1)


fraud['R1B'] = a['R1_x']/a['R1_y'] 
fraud['R2B'] = a['R2_x']/a['R2_y'] 
fraud['R3B'] = a['R3_x']/a['R3_y'] 
fraud['R4B'] = a['R4_x']/a['R4_y'] 
fraud['R5B'] = a['R5_x']/a['R5_y'] 
fraud['R6B'] = a['R6_x']/a['R6_y'] 
fraud['R7B'] = a['R7_x']/a['R7_y'] 
fraud['R8B'] = a['R8_x']/a['R8_y'] 
fraud['R9B'] = a['R9_x']/a['R9_y'] 


fraud = fraud.sort_index()
df = df.sort_index()

In [77]:
a = df.groupby(by = ['TAXCLASS']).agg({'R1':'mean', 'R2':'mean', 'R3':'mean', \
                             'R4':'mean', 'R5':'mean', 'R6':'mean',\
                             'R7':'mean', 'R8':'mean', 'R9':'mean'})

a = df.merge(a, how = 'left', on = 'TAXCLASS', suffixes = ['_x', '_y'])
a.index = range(1,len(a)+1)

fraud['R1T'] = a['R1_x']/a['R1_y'] 
fraud['R2T'] = a['R2_x']/a['R2_y'] 
fraud['R3T'] = a['R3_x']/a['R3_y'] 
fraud['R4T'] = a['R4_x']/a['R4_y'] 
fraud['R5T'] = a['R5_x']/a['R5_y'] 
fraud['R6T'] = a['R6_x']/a['R6_y'] 
fraud['R7T'] = a['R7_x']/a['R7_y'] 
fraud['R8T'] = a['R8_x']/a['R8_y'] 
fraud['R9T'] = a['R9_x']/a['R9_y'] 


fraud = fraud.sort_index()
df = df.sort_index()

In [78]:
#ZIP3
a = df.groupby(by = ['ZIP3']).agg({'R1':'mean', 'R2':'mean', 'R3':'mean', \
                             'R4':'mean', 'R5':'mean', 'R6':'mean',\
                             'R7':'mean', 'R8':'mean', 'R9':'mean'})

a = df.merge(a, how = 'left', on = 'ZIP3', suffixes = ['_x', '_y'])
a.index = range(1,len(a)+1)


fraud['R1Z3'] = a['R1_x']/a['R1_y'] 
fraud['R2Z3'] = a['R2_x']/a['R2_y'] 
fraud['R3Z3'] = a['R3_x']/a['R3_y'] 
fraud['R4Z3'] = a['R4_x']/a['R4_y'] 
fraud['R5Z3'] = a['R5_x']/a['R5_y'] 
fraud['R6Z3'] = a['R6_x']/a['R6_y'] 
fraud['R7Z3'] = a['R7_x']/a['R7_y'] 
fraud['R8Z3'] = a['R8_x']/a['R8_y'] 
fraud['R9Z3'] = a['R9_x']/a['R9_y'] 


fraud = fraud.sort_index()
df = df.sort_index()

In [79]:
#all

fraud['R1A'] = df['R1']/np.mean(df['R1'])

fraud['R2A'] = df['R2']/np.mean(df['R2'])
fraud['R3A'] = df['R3']/np.mean(df['R3'])
fraud['R4A'] = df['R4']/np.mean(df['R4'])
fraud['R5A'] = df['R5']/np.mean(df['R5'])

fraud['R6A'] = df['R6']/np.mean(df['R6'])
fraud['R7A'] = df['R7']/np.mean(df['R7'])
fraud['R8A'] = df['R8']/np.mean(df['R8'])

fraud['R9A'] = df['R9']/np.mean(df['R9'])

fraud = fraud.sort_index()
df = df.sort_index()


In [83]:
x=fraud.copy()
a = x.columns[0]
del x['RECORD']
del fraud['RECORD']
fraud=preprocessing.scale(fraud)
pca=PCA(n_components=8,copy=False)   # After plot the number of components and variance line chart, 8 is good
fraud=pca.fit_transform(fraud)#pca  
plt.plot(np.cumsum(pca.explained_variance_ratio_))

In [89]:
fraud=pd.DataFrame(preprocessing.scale(fraud),columns = ['component1', 'component2', 'component3', 
                                                             'component4', 'component5', 'component6', 
                                                             'component7', 'component8'])   

In [90]:
fraud.head()

,component1,component2,component3,component4,component5,component6,component7,component8
0,-0.018969,-0.094106,-0.074198,-0.029343,0.114940,0.021636,-0.112318,-0.056423
1,2.268693,16.572698,5.318490,-4.170378,-4.566852,4.891739,10.514436,-1.459022
2,-0.002922,0.079073,-0.013284,0.086232,0.021255,0.025630,-0.107896,0.057943
3,0.075090,-0.073002,-0.055368,0.012855,0.027145,-0.184669,-0.050301,0.126573
4,13.300791,-3.037439,-0.180367,-0.332434,-9.695408,-29.156924,9.289344,19.390790


In [91]:
cols = fraud.columns
fraud_backup = fraud.copy()
for i in cols:
    fraud[i] = fraud[i].apply(lambda x: x**2)

fraud['score']  = fraud.sum(axis = 1)
fraud['score'] = fraud['score'].apply(lambda x: x**0.5)
hello = fraud[['score']]
fraud = fraud_backup.copy()

In [95]:
## autoencoder model  --- takes time to run
components_train = fraud.values

input_dim = fraud.shape[1]
encoding_dim = 8



input_layer = Input(shape=(input_dim, ))
#encoder = Dense(encoding_dim, activation="elu", 
#               activity_regularizer=regularizers.l1(10e-5))(input_layer)    # for classification variables

encoder = Dense(int(encoding_dim / 2), activation="relu")(input_layer)
decoder = Dense(int(encoding_dim / 2), activation='elu')(encoder)
decoder = Dense(input_dim, activation='relu')(decoder)
autoencoder = Model(inputs=input_layer, outputs=decoder)


nb_epoch = 10
batch_size = 1024
autoencoder.compile(optimizer='adam', 
                    loss='mean_squared_error', 
                    metrics=['accuracy'])
checkpointer = ModelCheckpoint(filepath="model.h5",
                               verbose=0,
                               save_best_only=True)
tensorboard = TensorBoard(log_dir='./logs',
                          histogram_freq=0,
                          write_graph=True,
                          write_images=True)
history = autoencoder.fit(components_train, components_train,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    shuffle=True,
                   # validation_data=(X_test, X_test),
                    verbose=1,
                   callbacks=[checkpointer, tensorboard]
                    ).history

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
1070994/1070994 [==============================] - 2s 2us/step - loss: 0.9902 - acc: 0.3521
Epoch 2/10
 129024/1070994 [==>...........................] - ETA: 1s - loss: 1.4540 - acc: 0.3493

/anaconda3/lib/python3.7/site-packages/keras/callbacks.py:434: RuntimeWarning: Can save best model only with val_loss available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


1070994/1070994 [==============================] - 1s 1us/step - loss: 0.9365 - acc: 0.3453
Epoch 3/10
1070994/1070994 [==============================] - 1s 1us/step - loss: 0.8949 - acc: 0.3514
Epoch 4/10
1070994/1070994 [==============================] - 1s 1us/step - loss: 0.8576 - acc: 0.3787
Epoch 5/10
1070994/1070994 [==============================] - 1s 1us/step - loss: 0.8354 - acc: 0.3798
Epoch 6/10
1070994/1070994 [==============================] - 1s 1us/step - loss: 0.8205 - acc: 0.3607
Epoch 7/10
1070994/1070994 [==============================] - 1s 1us/step - loss: 0.8071 - acc: 0.3444
Epoch 8/10
1070994/1070994 [==============================] - 1s 1us/step - loss: 0.7939 - acc: 0.3365
Epoch 9/10
1070994/1070994 [==============================] - 1s 1us/step - loss: 0.7828 - acc: 0.3379
Epoch 10/10
1070994/1070994 [==============================] - 1s 1us/step - loss: 0.7688 - acc: 0.3328


In [96]:
components_train

array([[-1.89687140e-02, -9.41063287e-02, -7.41981491e-02, ...,
         2.16357537e-02, -1.12317902e-01, -5.64226292e-02],
       [ 2.26869289e+00,  1.65726980e+01,  5.31849009e+00, ...,
         4.89173896e+00,  1.05144361e+01, -1.45902231e+00],
       [-2.92216294e-03,  7.90728026e-02, -1.32838306e-02, ...,
         2.56295260e-02, -1.07896275e-01,  5.79433263e-02],
       ...,
       [-2.66936603e-02, -1.05955413e-01, -8.35575997e-03, ...,
        -1.10786274e-02, -1.22804235e-01,  5.32574743e-02],
       [-1.73046136e-02, -8.92742593e-02, -5.42809405e-03, ...,
        -9.81328516e-03, -1.05260673e-01,  6.18442107e-02],
       [-2.01080477e-02, -6.19584967e-02,  8.13475921e-03, ...,
         8.70429782e-03, -7.39934999e-02,  5.66006494e-02]])

In [97]:
predictions = autoencoder.predict(components_train)
predictions

array([[0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 1.1329055e-02],
       [0.0000000e+00, 1.5581633e+01, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 7.3858440e-02, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       ...,
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 6.7619056e-02],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 5.9738338e-02],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 1.2935460e-02]], dtype=float32)

In [98]:

predictions = pd.DataFrame.from_records(predictions, columns = ['component1', 'component2', 'component3', 
                                                             'component4', 'component5', 'component6', 
                                                             'component7', 'component8'] ) 
